In [1]:
# Computes stochastic closure certificate using SOS for stochastic systems in verifying LTL specifications

# include important libraries
using JuMP
using MosekTools
using DynamicPolynomials
using MultivariatePolynomials
using LinearAlgebra
using TSSOS # important for SOS, see https://github.com/wangjie212/TSSOS
using Distributions # for the noise

In [2]:
@polyvar x1 x2 y1 y2 x01 x02 # global vars used in monomials
vars = [x1, x2, y1, y2]

sos_tol = 1 # the maximum degree of unknown SOS polynomials = deg + sos_tol 
error = 2   # precision digit places
gamma = 0.07
lamda = 80
# p-sat = 99.91%
sig, mean = 0.01, 0

tau1, tau2, delta = .05, .05, 7 # S-procedure constants in condition 3

# define constants
alp  = 0.004
thet = 0.01
mu   = 0.15
Th   = 40
Te   = 0

# define temperature controller u(x) and u(y) symbolically
u_x1, u_x2 = 0.59 - 0.011 * x1, 0.59 - 0.011 * x2
u_y1, u_y2 = 0.59 - 0.011 * y1, 0.59 - 0.011 * y2

# build A symbolically
A11x1, A11y1 = 1 - 2*alp - thet - mu * u_x1, 1 - 2*alp - thet - mu * u_y1
A12 = alp
A21 = alp
A22x2, A22y2 = 1 - 2*alp - thet - mu * u_x2, 1 - 2*alp - thet - mu * u_y2 

# build 2rm-temp system dynamics f_sym(x, y)
f_symx1 = A11x1 * x1 + A12 * x2 + mu * Th * u_x1 + thet * Te
f_symx2 = A21 * x1 + A22x2 * x2 + mu * Th * u_x2 + thet * Te
f_symy1 = A11y1 * y1 + A12 * y2 + mu * Th * u_y1 + thet * Te
f_symy2 = A21 * y1 + A22y2 * y2 + mu * Th * u_y2 + thet * Te

# noise generator
function noise()
    return rand(Normal(mean, sqrt(sig)))
end

# creates the parametrized SCC
function add_paramC_poly!(model, var1, deg, q, p)
    basis = monomials(var1, 0:deg) # basis in x and y
    coeffs = @variable(model, [1:length(basis)], base_name="c_$(q)_$(p)")
    C_qp = sum(coeffs[i] * basis[i] for i in 1:length(basis))
    return C_qp, coeffs, basis
end

# creates the parametrized ranking functions
function add_paramV_poly!(model, var2, deg, q, p)
    basis = monomials(var2, 0:deg)
    coeffs = @variable(model, [1:length(basis)], base_name="v_$(q)_$(p)")
    V_qp = sum(coeffs[i] * basis[i] for i in 1:length(basis))
    return V_qp, coeffs, basis
end

# the considered NBA representing a_0 and GF(a1)
S = Dict(
    :"q0" => [(Set(["a0"]), :"q1"), (Set(["a3"]), :"q2"), (Set(["a1", "a2"]), :"q3")],
    :"q1" => [(Set(["a0", "a2"]), :"q1"), (Set(["a1", "a3"]), :"q2")],
    :"q2" => [(Set(["a0", "a2"]), :"q1"), (Set(["a1", "a3"]), :"q2")],
    :"q3" => [(Set(["a0", "a1", "a2", "a3"]), :"q3")]
)

Q0, QF = Set(["q0"]), Set(["q2"]) # Initial and Acceptance set of ~NBA

# considering that AP = {{a0}, {a1}, {~(a0 or a1)}={a2}, {a0 and a1}={a3}}
g_a01, g_a02 = [x1-21, 24-x1, x2-21, 24-x2], [x1-21, 24-x1, x2-21, 24-x2, y1-21, 24-y1, y2-21, 24-y2] # poly describing X0 = [21, 24], where L(x0) = a0
g_a11, g_a12 = [x1-20, 26-x1, x2-20, 26-x2], [x1-20, 26-x1, x2-20, 26-x2, y1-20, 26-y1, y2-20, 26-y2] # X_a1 = [20, 26]^2
g_a21, g_a22 = [x1-26, 34-x1, x2-26, 34-x2], [x1-26, 34-x1, x2-26, 34-x2, y1-26, 34-y1, y2-26, 34-y2] # X_a2 = [26, 34]^2
g_a31, g_a32 = g_a01, g_a02 # since X_a3 = X_a0

# for the first condition (x1, x2)
g1 = Dict(
    :"a0" => g_a01, :"a1" => g_a11, :"a2" => g_a21, :"a3" => g_a31
)
# for the 2nd condition
g2 = Dict(
    :"a0" => g_a02, :"a1" => g_a12, :"a2" => g_a22, :"a3" => g_a32
)     

# g(x0,x,y)
g3 = [x01-21, 24-x01, x02-21, 24-x02, x1-20, 34-x1, x2-20, 34-x2, y1-20, 34-y1, y2-20, 34-y2]

# polynomial stochastic closure certificate of degree deg and NBA S
function scc(deg)
    # synthesize SCC by using the standard formulation
    # deg: degree of scc template
    model = Model(optimizer_with_attributes(Mosek.Optimizer))
    set_optimizer_attribute(model, MOI.Silent(), true)

    C_dict = Dict()  # key => (symbolic_poly, numeric_poly)
    
    for q in keys(S)
        for (lab_set, qn) in S[q]
            for lab in lab_set
                C, Cc, Cb = add_paramC_poly!(model, vars, deg, q, qn) # CC, CC coefficients, and CC basis functions
                # these modification takes care of the expectation
                EC = C(vars=>[x1,x2,f_symx1 + noise(),f_symx2 + noise()]) 
                # see TSSOS https://github.com/wangjie212/TSSOS
                key = (q, qn, lab)  
                C_dict[key] = (C, Cc, Cb)  # store symbolic, coefficients, basis

                # 1st condition
                add_psatz!(model, -EC + gamma, [x1, x2], g1[lab], [], div(deg+sos_tol,2), QUIET=true, CS=false, TS=false, GroebnerBasis=true) 
                for p in keys(S)
                    C1, Cc1, Cb1 = add_paramC_poly!(model, vars, deg, p, qn)
                    EC1 = C1(vars=>[x1,x2,f_symy1 + noise(),f_symy2 + noise()]) 
                    C2, Cc2, Cb2 = add_paramC_poly!(model, vars, deg, p, q)
                    key1, key2 = (p, qn), (p, q)  
                    C_dict[key1], C_dict[key2] = (C1, Cc1, Cb1), (C2, Cc2, Cb2) 
                    
                    # 2nd condition
                    add_psatz!(model, -EC1 + C2, vars, g2[lab], [], div(deg+sos_tol,2), QUIET=true, CS=false, TS=false, GroebnerBasis=true) 
                end
            end
        end
    end
    for q0 in Q0
        for qf1 in QF
            V1, Vc1, Vb1 = add_paramV_poly!(model, [x01, x02, y1, y2], deg, q0, qf1)
            key3a = (q0, qf1, "V")  
            C_dict[key3a] = (V1, Vc1, Vb1)
            for qf2 in QF
                V2, Vc2, Vb2 = add_paramV_poly!(model, [x01, x02, x1, x2], deg, q0, qf2)
                C3, Cc3, Cb3 = add_paramC_poly!(model, [x01, x02, x1, x2], deg, q0, qf2)
                C4, Cc4, Cb4 = add_paramC_poly!(model, vars, deg, qf2, qf1)
                
                key3b = (q0, qf2, "V")  
                C_dict[key3b] = (V2, Vc2, Vb2)
                key4 = (q0, qf2)  
                C_dict[key4] = (C3, Cc3, Cb3)
                key5 = (qf1, qf2)  
                C_dict[key5] = (C4, Cc4, Cb4)
                # 3rd condition
                add_psatz!(model, -V1 + V2 - delta + tau1 * (lamda - C3) + tau2 * (lamda - C4), [x01, x02, x1, x2, y1, y2], g3, [], div(deg+sos_tol,2), QUIET=true, CS=false, TS=false, GroebnerBasis=true) 
            end
        end
    end
        
    optimize!(model) # solve for coefficients
    status = termination_status(model)
    C_eval_dict = Dict() # Get numerical values of coefficients and plug into polynomials
    for (key, (C, Cc, Cb)) in C_dict
        coeff_vals = round.(value.(Cc); digits=error)  # Round each coefficient to 2 decimal places
        C_numeric = sum(coeff_vals[i] * Cb[i] for i in eachindex(Cb))
        C_eval_dict[key] = (C, C_numeric)
    end
    
    # status might be optimal but if all Bc approx 10^{-error}, it's essentially 0 so OPTIMAL != CC.
    return status, C_eval_dict
end
        

# Simulation
file = open("./systems/2rm_temp_SCC_SOS.txt", "w")

deg = 1 # desired degree of SCC
stats = @timed (status, CC_data) = scc(deg)

write(file, "poly deg: "*string(deg)*"\n")
write(file, "status: "*string(status)*"\n")
write(file, "Number of SCC polynomials: "*string(length(CC_data))*"\n")
write(file, "time: "*string(stats.time)*"\n\n")

# Write the dictionary line-by-line
for (k, v) in CC_data
    write(file, "Key: $(k)\n")
    write(file, "Polynomial: $(v[1])\n")
    write(file, "Coefficients: $(v[2])\n\n")
end

close(file)

println("Finished")

Finished
